In [1]:
from pyspark.sql import SparkSession
from operator import add
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import corr, length, col, when
import time

#change spark.cores.max to change the total number of cores used. Keep executor cores set to 1 since the small vms have only one core having more here causes a ressource issue

spark_session = SparkSession\
        .builder\
        .master("spark://de-i-19:7077")\
        .appName("Project_Group19")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.executor.cores", 1)\
        .config("spark.cores.max", 4)\
        .config("spark.executor.memory","1g")\
        .config("spark.shuffle.service.enabled", "false")\
        .config("spark.dynamicAllocation.enabled", "false")\
        .getOrCreate()




# RDD API
spark_context = spark_session.sparkContext

# spark_context.setLogLevel("INFO")
spark_context.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/05 16:36:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/05 16:36:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Load Data

In [2]:
data = spark_session.read.csv("hdfs://192.168.2.246:9000/user/ubuntu/amazon_reviews_us_Baby_v1_00.tsv", sep="\t", header=True, inferSchema=True)
data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [3]:
data.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...|2015-08-31 00:00:00|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...| 

# Start Timer

In [4]:
start_time = time.time()

In [5]:
# Add the derived columns

data = data.withColumn("vine_numeric", when(col("vine") == "Y", 1).otherwise(0))
data = data.withColumn("verified_purchase_numeric", when(col("verified_purchase") == "Y", 1).otherwise(0))
data = data.withColumn("review_length", length(col("review_body")))
data = data.withColumn("total_votes_float", col("total_votes").cast("float"))
data = data.withColumn("helpful_votes_float", col("helpful_votes").cast("float"))

#data.printSchema()
#data.show(5)

# Calculate correlation
correlation_helpful = data.corr("review_length", "helpful_votes_float")
correlation_vine = data.corr("review_length", "vine_numeric")
correlation_star_rating = data.corr("review_length", "star_rating")
correlation_verified_purchase = data.corr("review_length", "verified_purchase_numeric")

print("Correlation between review length and helpful votes:", correlation_helpful)
print("Correlation between review length and vine:", correlation_vine)
print("Correlation between review length and star rating:", correlation_star_rating)
print("Correlation between review length and verified purchase:", correlation_verified_purchase)

Correlation between review length and helpful votes: 0.22581032092668252
Correlation between review length and vine: 0.13254068818960668
Correlation between review length and star rating: -0.09221360271671288
Correlation between review length and verified purchase: -0.26605155052081564


In [6]:
end_time = time.time()
duration = end_time - start_time
print("Experiment took", duration)

Experiment took 34.44105100631714


In [8]:
spark_session.stop()